# ICRS Implementation
That is, for applications of this accuracy, the distinctions between the ICRS, FK5,
and dynamical equator and equinox of J2000.0 are not significant.

A matrix B is required to convert ICRS data to the dynamical mean equator and equinox of J2000.0
(the “J2000.0 system”),the latter considered for this purpose to be a barycentric system.

In barycentric case, $r_{mean(2000)} = B\ r_{ICRS}$

$$ r=\left(\begin{array}{c}
    \cos\delta\cos\alpha\\
    \cos\delta\sin\alpha\\
    \sin\delta\end{array}\right) $$
    
In geocentric case, $r_{mean(2000)} = B\ r_{GCRS}$

The matrix B is, to first order,

$$ B=\left(\begin{array}{ccc}
    1 & d\alpha_0 & -\epsilon_0\\
    -d\alpha_0 &1 & -\eta_0\\
    \epsilon_0 &\eta_0 &1\end{array}\right) $$
where $d\alpha_0=-14.6$ mas, $\epsilon_0=-16.6170$ mas, $\eta_0=-6.8192$ mas, all converted to radians
(divide by 206264806.247).

Although the above matrix is adequate for most applications, a more precise result can be
obtained by using the second-order version:
$$ B=\left(\begin{array}{ccc}
    1-\frac{1}{2}(d\alpha_0^2+\epsilon_0^2) & d\alpha_0 & -\epsilon_0\\
    -d\alpha_0-\eta_0\epsilon_0 &1-\frac{1}{2}(d\alpha_0^2+\eta_0^2) & -\eta_0\\
    \epsilon_0-\eta_0d\alpha_0 &\eta_0+\epsilon_0d\alpha_0 &1-\frac{1}{2}(\eta_0^2+\epsilon_0^2)\end{array}\right) $$

The above matrix, from Slabinski (2005), is an excellent approximation to the set of rotations
$R\mbox{_}x(−\eta_0)\mbox{R_y}(\epsilon_0)\mbox{R_z}(d\alpha_0)$.

    

In [71]:
from skyfield.api import load
import numpy as np
def R_x(angle):
    v1=[1.,0.,0.]
    v2=[0.,np.cos(angle),np.sin(angle)]
    v3=[0.,-np.sin(angle),np.cos(angle)]
    return(np.vstack((v1,v2,v3)))
def R_y(angle):
    v2=[0.,1.,0.]
    v3=[np.sin(angle),0.,np.cos(angle)]
    v1=[np.cos(angle),0.,-np.sin(angle)]
    return(np.vstack((v1,v2,v3)))
def R_z(angle):
    v3=[0.,0.,1.]
    v1=[np.cos(angle),np.sin(angle),0.]
    v2=[-np.sin(angle),np.cos(angle),0.]
    return(np.vstack((v1,v2,v3)))
def polar2cart(lonlat):
    lon=lonlat[0]
    lat=lonlat[1]
    phi=np.pi/2-lat
    z=np.cos(phi)
    x=np.sin(phi)*np.cos(lon)
    y=np.sin(phi)*np.sin(lon)
    return(np.array([x,y,z]))
def cart2polar(v):
    v=v/np.linalg.norm(v)
    lon=np.arctan2(v[1],v[0])
    lat=np.pi/2-np.arccos(v[2])
    return(np.array([lon,lat]))
ts = load.timescale()
t = ts.utc(2016,9,8,10,23,30)
B=t.PT.dot(t.NT).dot(t.M)
da0=-14.6/206264806.247
epsilon0=-16.6170/206264806.247
eta0=-6.8192/206264806.247
B1=np.matrix([[1,da0,-epsilon0],[-da0,1,-eta0],[epsilon0,eta0,1 ]])
B2=np.matrix([[1-0.5*(da0**2+epsilon0**2),da0,-epsilon0],
              [-da0-eta0*epsilon0,1-0.5*(da0**2+eta0**2),-eta0],
              [epsilon0-eta0*da0,eta0+epsilon0*da0,1-0.5*(eta0**2+epsilon0**2) ]])
B3=R_x(-eta0).dot(R_y(epsilon0)).dot(R_z(da0))
print(B)
print(B1)
print(B2)
print(B3)
print(B-B1)
print(B-B2)
print(B-B3)
print(B2-B3)

[[  1.00000000e+00  -7.07827974e-08   8.05614894e-08]
 [  7.07827974e-08   1.00000000e+00   3.30604145e-08]
 [ -8.05614894e-08  -3.30604145e-08   1.00000000e+00]]
[[  1.00000000e+00  -7.07827974e-08   8.05614894e-08]
 [  7.07827974e-08   1.00000000e+00   3.30604145e-08]
 [ -8.05614894e-08  -3.30604145e-08   1.00000000e+00]]
[[  1.00000000e+00  -7.07827974e-08   8.05614894e-08]
 [  7.07827948e-08   1.00000000e+00   3.30604145e-08]
 [ -8.05614917e-08  -3.30604088e-08   1.00000000e+00]]
[[  1.00000000e+00  -7.07827974e-08   8.05614894e-08]
 [  7.07827948e-08   1.00000000e+00   3.30604145e-08]
 [ -8.05614917e-08  -3.30604088e-08   1.00000000e+00]]
[[ -5.88418203e-15   1.19656639e-19  -1.28351961e-19]
 [  7.44422847e-19  -3.10862447e-15  -7.44115135e-18]
 [  1.28351961e-19  -7.40564215e-18  -3.99680289e-15]]
[[ -1.11022302e-16   1.19656639e-19  -1.28351961e-19]
 [  2.66414066e-15  -1.11022302e-16  -7.44115135e-18]
 [  2.34023698e-15  -5.70977323e-15  -2.22044605e-16]]
[[ -1.11022302e-16   1

In [64]:
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5,GCRS  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

In [74]:
c = SkyCoord(10, 20, unit="deg",frame='gcrs')
print(c)
c_J2000=c.transform_to(GCRS())
print(c_J2000)
c_cartesian=np.array([c.cartesian.x.value,c.cartesian.y.value,c.cartesian.z.value])
c_J2000_cartesian=np.array([c_J2000.cartesian.x.value,c_J2000.cartesian.y.value,c_J2000.cartesian.z.value])
print(c_cartesian)
print(c_J2000_cartesian)
print(B.dot(c_cartesian))

<SkyCoord (GCRS: obstime=J2000.000, obsgeoloc=[ 0.  0.  0.] m, obsgeovel=[ 0.  0.  0.] m / s): (ra, dec) in deg
    (10.0, 20.0)>
<SkyCoord (GCRS: obstime=J2000.000, obsgeoloc=[ 0.  0.  0.] m, obsgeovel=[ 0.  0.  0.] m / s): (ra, dec) in deg
    (10.0, 20.0)>
[ 0.92541658  0.16317591  0.34202014]
[ 0.92541658  0.16317591  0.34202014]
[ 0.92541659  0.16317599  0.34202006]


In [60]:
import astropy.coordinates as coord
dir(coord)

['AltAz',
 'Angle',
 'BarycentricTrueEcliptic',
 'BaseCoordinateFrame',
 'BaseRepresentation',
 'BoundsError',
 'CIRS',
 'CartesianRepresentation',
 'CompositeTransform',
 'ConvertError',
 'CoordinateAttribute',
 'CoordinateTransform',
 'CylindricalRepresentation',
 'Distance',
 'DynamicMatrixTransform',
 'EarthLocation',
 'EarthLocationAttribute',
 'FK4',
 'FK4NoETerms',
 'FK5',
 'FrameAttribute',
 'FunctionTransform',
 'GCRS',
 'Galactic',
 'Galactocentric',
 'GenericFrame',
 'GeocentricTrueEcliptic',
 'HCRS',
 'HeliocentricTrueEcliptic',
 'ICRS',
 'ITRS',
 'IllegalHourError',
 'IllegalHourWarning',
 'IllegalMinuteError',
 'IllegalMinuteWarning',
 'IllegalSecondError',
 'IllegalSecondWarning',
 'Latitude',
 'Longitude',
 'PhysicsSphericalRepresentation',
 'PrecessedGeocentric',
 'QuantityFrameAttribute',
 'RangeError',
 'RepresentationMapping',
 'SkyCoord',
 'SkyOffsetFrame',
 'SphericalRepresentation',
 'StaticMatrixTransform',
 'Supergalactic',
 'TimeFrameAttribute',
 'TransformGra

<SkyCoord (ICRS): (ra, dec) in deg
    (10.68458, 41.26917)>
<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (10.68459154, 41.26917146)>


## Verify transform from ICRS to FK5

In [80]:
da0=-22.9/206264806.247
epsilon0=9.1/206264806.247
eta0=-19.9/206264806.247
B1_fk5=np.array([[1,da0,-epsilon0],[-da0,1,-eta0],[epsilon0,eta0,1 ]])
B2_fk5=np.array([[1-0.5*(da0**2+epsilon0**2),da0,-epsilon0],
              [-da0-eta0*epsilon0,1-0.5*(da0**2+eta0**2),-eta0],
              [epsilon0-eta0*da0,eta0+epsilon0*da0,1-0.5*(eta0**2+epsilon0**2) ]])
B3_fk5=R_x(-eta0).dot(R_y(epsilon0)).dot(R_z(da0))

c_icrs = SkyCoord(ra=10.68458*u.degree, dec=41.26917*u.degree, frame='icrs')
c_fk5 = c_icrs.transform_to('fk5')
print(c_icrs)
print(c_fk5)
c_icrs_cartesian=np.array([c_icrs.cartesian.x.value,c_icrs.cartesian.y.value,c_icrs.cartesian.z.value])
c_fk5_cartesian=np.array([c_fk5.cartesian.x.value,c_fk5.cartesian.y.value,c_fk5.cartesian.z.value])
print(c_icrs_cartesian)
print(c_fk5_cartesian)
print(B1_fk5.dot(c_icrs_cartesian))
print(B2_fk5.dot(c_icrs_cartesian))
print(B3_fk5.dot(c_icrs_cartesian))
print(B1_fk5.dot(c_icrs_cartesian)-c_fk5_cartesian)
print(B2_fk5.dot(c_icrs_cartesian)-c_fk5_cartesian)
print(B3_fk5.dot(c_icrs_cartesian)-c_fk5_cartesian)

<SkyCoord (ICRS): (ra, dec) in deg
    (10.68458, 41.26917)>
<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (10.68459154, 41.26917146)>
[ 0.73858814  0.13935181  0.65959733]
[ 0.73858809  0.13935196  0.65959735]
[ 0.73858809  0.13935196  0.65959735]
[ 0.73858809  0.13935196  0.65959735]
[ 0.73858809  0.13935196  0.65959735]
[  5.21804822e-15  -1.63757896e-15   1.23234756e-14]
[ 0.  0.  0.]
[ -1.11022302e-16  -2.77555756e-17   0.00000000e+00]


## Ephemerides of the Major Solar System Bodies
### The JPL Ephemerides
The position and velocity data provided by the JPL ephemerides represent the **centers of mass**
of each planet-satellite system (although data for the Earth and Moon can be extracted separately).
Therefore, the positions, when converted to geocentric apparent places — angular coordinates as
seen from Earth — do not precisely indicate the **center of the apparent planetary disk**. Displacements
can amount to a few tens of milliarcseconds for Jupiter and Saturn, a few milliarcseconds
for Uranus and Neptune, and about 0.1 arcsecond for Pluto.

In the context of traditional equatorial celestial coordinate systems, the adjective **mean** is applied
to quantities (pole, equator, equinox, coordinates) affected only by precession, while **true** describes
quantities affected by both precession and nutation. This is a computational distinction only, since
precession and nutation are simply different aspects of the same physical phenomenon. Thus, it is
the true quantities that are directly relevant to observations; mean quantities now usually represent 
an intermediate step in the computations, or the final step where only very low accuracy is needed
(10 arcseconds or worse) and nutation can be ignored.

Mathematically, this sequence can be represented as follows:
$$r_{true}(t) = N(t) P(t) B r_{GCRS}$$

In [84]:
def P(T):
    epsilon0=84381.406
    phiA=5038.481507 *T - 1.0790069 *T**2 - 0.00114045* T**3 + 0.000132851* T**4 - 0.0000000951* T**5
    omegaA=epsilon0 - 0.025754 *T + 0.0512623 *T**2 - 0.00772503 *T**3 - 0.000000467 *T**4 + 0.0000003337 *T**5
    chiA=10.556403 *T - 2.3814292 *T**2 - 0.00121197 *T**3 + 0.000170663 *T**4 - 0.0000000560 *T**5
    return(R_z(chiA/206264.806247).dot(R_x(-omegaA/206264.806247)).dot(R_z(-phi1/206264.806247)).dot(R_x(epsilon0/206264.806247)))